<a href="https://colab.research.google.com/github/tada20001/NLP_2023/blob/main/CH18_07_KoBERT%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EA%B8%B0%EA%B3%84%EB%8F%85%ED%95%B4(Machine_Reading_Comprehension).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.3 MB/s eta 0:00:00


### 1. 데이터 전처리

한국어 질의응답(Question Answering) 데이터셋인 KorQuad 데이터를 다운로드함

In [2]:
import os
import json
import numpy as np
from tqdm import tqdm
from pathlib import Path
from transformers import BertTokenizerFast
import tensorflow as tf

!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json -O KorQuAD_v1.0_train.json
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json -O KorQuAD_v1.0_dev.json

--2023-05-22 07:42:25--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘KorQuAD_v1.0_train.json’

KorQuAD_v1.0_train. 100%[===================>]  36.74M   185MB/s    in 0.2s    

2023-05-22 07:42:25 (185 MB/s) - ‘KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2023-05-22 07:42:25--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to: ‘KorQuAD_v1.0_dev.json’

KorQuAD_v1.0_dev.js 100%[======

In [3]:
# 크기 확인
def read_squad(path):
  path = Path(path)
  with open(path, 'rb') as f:
    squad_dict = json.load(f)
  contexts = []
  questions = []
  answers = []
  for group in squad_dict['data']:
    for passage in group['paragraphs']:
      context = passage['context']
      for qa in passage['qas']:
        question = qa['question']
        for answer in qa['answers']:
          contexts.append(context)
          questions.append(question)
          answers.append(answer)
  return contexts, questions, answers

In [4]:
train_contexts, train_questions, train_answers = read_squad('KorQuAD_v1.0_train.json')

In [5]:
val_contexts, val_questions, val_answers = read_squad('KorQuAD_v1.0_dev.json')

In [6]:
print('훈련데이터 본문 갯수:', len(train_contexts))
print('훈련데이터 질문 갯수:', len(train_questions))
print('훈련데이터 답변 갯수:', len(train_answers))
print('테스트 데이터 본문 갯수:', len(val_contexts))
print('테스트 데이터 질문 갯수:', len(val_questions))
print('테스트 데이터 답변 갯수:', len(val_answers))

훈련데이터 본문 갯수: 60407
훈련데이터 질문 갯수: 60407
훈련데이터 답변 갯수: 60407
테스트 데이터 본문 갯수: 5774
테스트 데이터 질문 갯수: 5774
테스트 데이터 답변 갯수: 5774


In [7]:
# 첫번째 샘플 구성 확인
train_contexts[0]

'1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.'

In [8]:
train_questions[0]

'바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'

In [9]:
train_answers[0]  #정답과 정답위치 인덱스, 종료 인덱스는 없기 때문에, 종료인덱스도 추가 필요

{'text': '교향곡', 'answer_start': 54}

In [10]:
def add_end_idx(answers, contexts):
  for answer, context in zip(answers, contexts):
    # 뒤에 공백이 있으면 제거.
    answer['text'] = answer['text'].rstrip()  # 해당 텍스트의 오른쪽 공백부분 제거

    # 시작 인덱스에 정답길이를 더하여 종료 인덱스 계산
    gold_text = answer['text']
    start_idx = answer['answer_start']
    end_idx = start_idx + len(gold_text)

    assert context[start_idx:end_idx] == gold_text, 'end_index 계산에 에러가 있습니다.'
    answer['answer_end'] = end_idx

In [11]:
# 해당 함수를 통해 훈련데이터와 테스트 데이터 모두 종료 인덱스 추가
add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [12]:
# 확인
train_answers[0]

{'text': '교향곡', 'answer_start': 54, 'answer_end': 57}

In [13]:
train_contexts[0][54:57]

'교향곡'

In [14]:
# 본문과 질문 두가지에 대해 토크나이저 입력 데이터로 사용
tokenizer = BertTokenizerFast.from_pretrained('klue/bert-base')
train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

In [15]:
# 토큰화 결과 확인 : 본문 토큰화 결과와 질문 토큰화 결과가 연결되어 저장되어 있고, 두개는 SEP으로 구분됨
print(train_encodings[0].tokens)

['[CLS]', '183', '##9', '##년', '바그너', '##는', '괴테', '##의', '파우', '##스트', '##을', '처음', '읽', '##고', '그', '내용', '##에', '마음', '##이', '끌려', '이를', '소재', '##로', '해서', '하나', '##의', '교향곡', '##을', '쓰', '##려', '##는', '뜻', '##을', '갖', '##는', '##다', '.', '이', '시기', '바그너', '##는', '183', '##8', '##년', '##에', '빛', '독촉', '##으로', '산전', '##수', '##전', '##을', '다', '[UNK]', '상황', '##이', '##라', '좌절', '##과', '실망', '##에', '가득', '##했', '##으며', '메', '##피스', '##토', '##펠', '##레스', '##를', '만나', '##는', '파우', '##스트', '##의', '심경', '##에', '공감', '##했', '##다고', '한다', '.', '또한', '파리', '##에서', '아', '##브', '##네', '##크', '##의', '지휘', '##로', '파리', '음악', '##원', '관현', '##악단', '##이', '연주', '##하', '##는', '베토벤', '##의', '교향곡', '9', '##번', '##을', '듣', '##고', '깊', '##은', '감명', '##을', '받', '##았', '##는데', ',', '이것', '##이', '이듬해', '1', '##월', '##에', '파우', '##스트', '##의', '서', '##곡', '##으로', '쓰여진', '이', '작품', '##에', '조금', '##이', '##라도', '영향', '##을', '끼쳤', '##으리', '##라는', '것', '##은', '의심', '##할', '여지', '##가', '없', '##다', '.', '여기', '##의', '

첫번째 샘플 길이 확인!!

In [16]:
len(train_encodings[0].tokens)

512

첫번째 샘플 길이는 512로, 이는 BERT에서 입력을 받을 수 있는 최대길이가 512이기 때문임. 본문과 질문의 길이 합이 512보다 길 경우에는 본문이 잘릴 수 있고, 짧은 경우는 [PAD] 토큰을 추가하여 맞춰줌. 

첫번째 샘플의 정수인코딩 결과 확인!!

In [17]:
print(train_encodings[0].ids)

[2, 13934, 2236, 2440, 27982, 2259, 21310, 2079, 11994, 3791, 2069, 3790, 1508, 2088, 636, 3800, 2170, 3717, 2052, 9001, 8345, 4642, 2200, 3689, 3657, 2079, 19282, 2069, 1363, 2370, 2259, 936, 2069, 554, 2259, 2062, 18, 1504, 4342, 27982, 2259, 13934, 2196, 2440, 2170, 1195, 23260, 6233, 17370, 2113, 2165, 2069, 809, 1, 3706, 2052, 2181, 8642, 2145, 7334, 2170, 4983, 2371, 4007, 1065, 5917, 2386, 2559, 4443, 2138, 4026, 2259, 11994, 3791, 2079, 15864, 2170, 5487, 2371, 4683, 3605, 18, 3819, 5986, 27135, 1376, 2645, 2203, 2292, 2079, 5872, 2200, 5986, 4152, 2252, 22835, 16706, 2052, 5485, 2205, 2259, 17087, 2079, 19282, 29, 2517, 2069, 881, 2088, 652, 2073, 23404, 2069, 1122, 2886, 13964, 16, 3982, 2052, 9944, 21, 2429, 2170, 11994, 3791, 2079, 1258, 2465, 6233, 24294, 1504, 3967, 2170, 4027, 2052, 5121, 3979, 2069, 18274, 21575, 23548, 575, 2073, 5292, 2085, 7251, 2116, 1415, 2062, 18, 3776, 2079, 942, 2286, 2446, 4196, 2079, 3640, 6509, 636, 2079, 4450, 2170, 10329, 1513, 2259, 575, 7

In [18]:
# 어텐션 마스크 확인 ==> 뒤의 0은 [PAD]에 해
print(train_encodings[0].attention_mask)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [19]:
# 토큰에 실제 단어 위치정보 만들기
def add_token_positions(encodings, answers):
  start_positions = []
  end_positions = []
  deleting_list = []
  for i in tqdm(range(len(answers))):
    # 토큰화 전 문자의 인덱스(start, end)로부터 토큰화된 후의 문자의 인덱스 찾아냄
    start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
    end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

    # 시작 인덱스가 비정상인 경우. 즉 본문에 정답이 없는 경우
    if start_positions[-1] is None:
      start_positions[-1] = tokenizer.model_max_length
      deleting_list.append(i)
    
    # 종료 인덱스가 비정상인 경우, 즉 본문에 정답이 없는 경우
    if end_positions[-1] is None:
      end_positions[-1] = tokenizer.model_max_length
      if i not in deleting_list:
        deleting_list.append(i)
  encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
  return deleting_list

위 함수는 토큰화 전 정답의 인덱스였던 answer_start와 answer_end로부터 BERT 토크나이저로 토큰화한 후 인덱스를 찾아내고, start_positions와 end_positions에 저장함. 

char_to_token 모듈은 기본적으로 토큰화 이전의 문자 인덱스를 입력하면, 토큰화 후의 해당 문자의 인덱스를 추적해서 알려줌. 여기에서는 start_positions[-1]의 결과로 정답이 있는지 확인하는 알고리즘임. 

None을 리턴할 경우 deleting_list에 해당 샘플 인덱스를 저장해둠. 

In [20]:
deleting_list_for_train = add_token_positions(train_encodings, train_answers)
deleting_list_for_val = add_token_positions(val_encodings, val_answers)

100%|██████████| 5774/5774 [00:00<00:00, 279739.77it/s]


In [21]:
print(deleting_list_for_train)

[711, 726, 728, 729, 761, 765, 767, 768, 805, 2586, 2587, 2722, 2724, 2725, 2731, 3392, 3475, 3478, 3491, 3495, 3498, 3919, 4462, 4465, 4513, 4515, 4565, 4765, 4766, 4772, 4774, 4779, 5334, 6603, 6638, 6639, 6748, 6749, 6750, 6765, 6766, 6771, 6776, 6897, 6898, 6900, 7739, 7741, 9203, 9211, 10880, 11039, 11212, 11727, 11776, 11788, 11789, 11791, 12168, 13708, 13711, 13996, 14460, 14461, 14491, 14724, 14729, 14885, 15764, 15970, 15971, 15973, 15974, 15976, 15977, 15979, 15980, 16080, 17683, 17815, 17828, 18389, 18392, 19045, 19052, 19053, 19195, 19636, 19637, 19638, 19640, 19656, 19761, 19764, 19765, 20614, 20618, 21224, 21243, 21334, 21335, 21338, 21361, 21521, 21522, 22627, 22633, 24003, 24577, 24579, 24580, 24768, 25108, 25176, 25182, 25185, 25186, 25187, 25188, 25448, 25451, 25454, 25457, 25460, 27105, 27112, 27113, 27114, 27159, 27293, 27295, 27555, 27558, 28025, 28438, 28779, 29162, 29189, 29289, 29290, 29855, 31889, 31890, 31891, 31894, 31905, 32050, 32051, 32057, 32058, 32059, 3

In [22]:
len(deleting_list_for_train), len(deleting_list_for_val)

(267, 57)

위 샘플들은 본문에 정답이 없는 경우이므로 학습에서 제외함. 실제 본문에 정답이 없는 샘플 761번으로 확인

In [23]:
print(train_questions[761])
print('-'*50)
print(train_contexts[761])

소치 팀추월 파이널D에서 여자 팀추월 대표팀의 최종 성적은?
--------------------------------------------------
2014년 2월 9일 러시아 소치 아들레르 아레나서 열린 소치 동계올림픽 3000m 부문에서 김보름(21)은 4분12초08의 기록으로로 13위를 차지했다. 이날 3조로 경기를 치른 김보름은 21초05로 200m 구간을 통과한 후 2분31초34로 1800m 구간을 지났다. 이후 2200m 구간 통과 순간부터 스피드를 올리며 후반 들어 스퍼트를 올렸고, 결국 하위권이 아닌 중위권 기록을 남겼다. 김보름의 순위인 13위는 지난 2006 토리노 올림픽, 2010 밴쿠버 올림픽 당시 노선영(25-강원도청)이 기록한 19위를 넘어 한국 여자 3000m 부문의 가장 높은 순위다. 5조의 노선영은 4분19초02를 기록했다. 노선영은 200m 구간에서 21초32의 기록으로 지난 이후 속도를 올리지 못한 채로 결승점을 통과했다. 결국 노선영은 전체 26위의 성적을 남기며 경기를 마쳤다. 6조에서 경기를 소화한 양신영(24-전북도청)은 4분23초67을 기록해 이날 대회를 뛴 28명 중 최저의 기록을 남겼다. 한편 이날 대회는 4분00초34의 이레네 부스트(네덜란드)가 2006년 토리노 동계올림픽 이후 8년 만에 금메달을 다시 가져갔다. 대회 2연패를 노린 2위 마르티나 사블리코바(체코-4분01초95)와 3위 올가 그라프(러시아-4분03초47)에 앞선 기록이다. 16일 열린 1500m에서는 네덜란드의 요리엔 테르모르스가 1분53초51의 올림픽 기록으로 금메달을 차지했다. 은메달과 동메달도 네덜란드 선수들이 휩쓸었다. 은메달은 이레인 뷔스트(1분54초09)에게 돌아갔고 동메달은 하를로터 판바이크(1분54초54)가 주인이 됐다. 심지어 4위도 네덜란드 선수인 마리트 리엔스트라(1분56초40)가 차지했다. 김보름은 1분59초78로 21위에 올랐다. 노선영(25-강원도청)은 2분01초07로 29위, 양신영(24-전북도청)은 2분04초13으

In [24]:
print(train_answers[761])

{'text': '8위', 'answer_start': 1050, 'answer_end': 1052}


In [25]:
train_contexts[761][1050:1052]

'8위'

'8위'라는 단어가 BERT의 입력으로 사용하기 위해 토큰화된 결과에 존재하는지 확인!!

In [26]:
print(tokenizer.decode(train_encodings['input_ids'][761]))

[CLS] 2014년 2월 9일 러시아 소치 아들레르 아레나서 열린 소치 동계올림픽 3000m 부문에서 김보름 ( 21 ) 은 4분12초08의 기록으로로 13위를 차지했다. 이날 3조로 경기를 치른 김보름은 21초05로 200m 구간을 통과한 후 2분31초34로 1800m 구간을 지났다. 이후 2200m 구간 통과 순간부터 스피드를 올리며 후반 들어 스퍼트를 올렸고, 결국 하위권이 아닌 중위권 기록을 남겼다. 김보름의 순위인 13위는 지난 2006 토리노 올림픽, 2010 밴쿠버 올림픽 당시 노선영 ( 25 - 강원도청 ) 이 기록한 19위를 넘어 한국 여자 3000m 부문의 가장 높은 순위다. 5조의 노선영은 4분19초02를 기록했다. 노선영은 200m 구간에서 21초32의 기록으로 지난 이후 속도를 올리지 못한 채로 결승점을 통과했다. 결국 노선영은 전체 26위의 성적을 남기며 경기를 마쳤다. 6조에서 경기를 소화한 양신영 ( 24 - 전북도청 ) 은 4분23초67을 기록해 이날 대회를 뛴 28명 중 최저의 기록을 남겼다. 한편 이날 대회는 4분00초34의 이레네 부스트 ( 네덜란드 ) 가 2006년 토리노 동계올림픽 이후 8년 만에 금메달을 다시 가져갔다. 대회 2연패를 노린 2위 마르티나 사블리코바 ( 체코 - 4분01초95 ) 와 3위 올가 그라프 ( 러시아 - 4분03초47 ) 에 앞선 기록이다. 16일 열린 1500m에서는 네덜란드의 요리엔 테르모르스가 1분53초51의 올림픽 기록으로 금메달을 차지했다. 은메달과 동메달도 네덜란드 선수들이 휩쓸었다. 은메달은 이레인 뷔스트 ( 1분54초09 ) 에게 돌아갔고 동메달은 하를로터 판바이크 ( 1분54초54 ) 가 주인이 됐다. 심지어 4위도 네덜란드 선수인 마리트 리엔스트라 ( 1분56초40 ) 가 차지했다. 김보름은 1분59초78로 21위에 올랐다. 노선영 ( 25 - 강원도청 ) 은 2분01초07로 29위, 양신영 ( 24 - 전북도청 ) [SEP] 소치 팀추월 파이널D에서 여자 팀추월 대표팀의 최

길이 512로 맞추는 토큰화 과정에서 정답 부분이 짤려 답을 찾을 수 없게 됨. 따라서 이 부분은 제거 대상임

In [27]:
def delete_samples(encodings, deleting_list):
  input_ids = np.delete(np.array(encodings['input_ids']), deleting_list, axis=0)
  attention_masks = np.delete(np.array(encodings['attention_mask']), deleting_list, axis=0)
  start_positions = np.delete(np.array(encodings['start_positions']), deleting_list, axis=0)
  end_positions = np.delete(np.array(encodings['end_positions']), deleting_list, axis=0)

  X_data = [input_ids, attention_masks]
  y_data = [start_positions, end_positions]

  return X_data, y_data

In [28]:
X_train, y_train = delete_samples(train_encodings, deleting_list_for_train)
X_test, y_test = delete_samples(val_encodings, deleting_list_for_val)

In [29]:
print('------------------삭제 전------------------')
print('훈련데이터 샘플 수 :', len(train_contexts))
print('테스트 데이터 샘플 수 :', len(val_contexts))
print('------------------삭제 후------------------')
print('훈련데이터 샘플 수 :', len(X_train[0]))
print('테스트 데이터 샘플 수 :', len(X_test[0]))

------------------삭제 전------------------
훈련데이터 샘플 수 : 60407
테스트 데이터 샘플 수 : 5774
------------------삭제 후------------------
훈련데이터 샘플 수 : 60140
테스트 데이터 샘플 수 : 5717


In [30]:
len(deleting_list_for_train), len(deleting_list_for_val)

(267, 57)

### 2. BERT를 이용한 Question Answering

TPU 사용 코드 !!

In [31]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

기계 독해를 위해 BERT가 사용하는 출력층은 두개임. 각각 정답의 시작 인덱스와 종료 인덱스에 해당됨. 만약 BERT가 입력받는 입력길이가 512라면, 총 512개 후보 중에서 시작 인덱스를 예측하는 다중 클래스 분류문제와 총 512개 후보 중에서 종료인덱스를 예측하는 다중클래스 분류문제, 이렇게 총 두개의 문제를 풀게됨


그리고 모델이 각각 시작 인덱스와 종료 인덱스를 예측하도록 학습하게 되면, 이를 가지고 임의의 본문과 질문이 입력되면, 테스트 단계에서 모델은 본문으로부터 정답의 시작 인덱스와 종료 인덱스를 예측함

In [32]:
from transformers import TFBertModel

In [33]:
class TFBertForQuestionAnswering(tf.keras.Model):
  def __init__(self, model_name):
    super(TFBertForQuestionAnswering, self).__init__()
    self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
    self.qa_outputs = tf.keras.layers.Dense(2, kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02), name='qa_outputs')
    self.softmax = tf.keras.layers.Activation(tf.keras.activations.softmax)

  def call(self, inputs):
    input_ids, attention_mask = inputs
    outputs = self.bert(input_ids, attention_mask=attention_mask)

    # BERT의 마지막 층의 모든 토큰들
    sequence_output = outputs[0]

    logits = self.qa_outputs(sequence_output)

    # 사용할 출력층은 총 2개, 각각 시작 인덱스 예측과 종료인덱스 예측에 사용됨
    start_logits, end_logits = tf.split(logits, 2, axis=-1)

    # start_logits = (batch_size, sequence_length,)
    # end_logits = (batch_size, sequence_length,)
    start_logits = tf.squeeze(start_logits, axis=-1)
    end_logits = tf.squeeze(end_logits, axis=-1)

    # 시작 인덱스에 대한 다중 클래스 분류 문제
    start_probs = self.softmax(start_logits)

    # 종료 인덱스에 대한 다중 클래스 분류 문제
    end_probs = self.softmax(end_logits)

    return start_probs, end_probs

In [34]:
with strategy.scope():
  model = TFBertForQuestionAnswering("klue/bert-base")
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
  model.compile(optimizer=optimizer, loss=[loss, loss])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

배치 크기는 16, 총 에포크 3을 학습함. 정답의 시작 인덱스와 종료 인덱스에 대한 오차를 각각 계산하므로 에포크 동안 관찰되는 오차는 2가지임. 각각 output_1_loss와 output_2_loss가 출력됨

In [35]:
history = model.fit(X_train, y_train, epochs=3, verbose=1, batch_size=16)

Epoch 1/3


3759/3759 [==============================] - 549s 114ms/step - loss: 1.1321 - output_1_loss: 0.5066 - output_2_loss: 0.6255
Epoch 2/3
3759/3759 [==============================] - 434s 116ms/step - loss: 0.6190 - output_1_loss: 0.2621 - output_2_loss: 0.3568
Epoch 3/3
3759/3759 [==============================] - 430s 114ms/step - loss: 0.4513 - output_1_loss: 0.1852 - output_2_loss: 0.2661


### 3. 테스트 데이터에 대한 예측

predict_test_data_by_idx 함수는 임의의 인덱스를 넣으면 해당 인덱스의 테스트 데이터 샘플을 모델 입력으로 넣어 결과를 출력하는 함수임

In [36]:
def predict_test_data_by_idx(idx):
  context = tokenizer.decode(X_test[0][idx]).split('[SEP] ')[0]
  question = tokenizer.decode(X_test[0][idx]).split('[SEP] ')[1]
  print('본문 :', context)
  print('질문 :', question)
  answer_encoded = X_test[0][idx][y_test[0][idx]:y_test[1][idx]+1]
  print('정답 :', tokenizer.decode(answer_encoded))

  output = model([tf.constant(X_test[0][idx])[None, :], tf.constant(X_test[1][idx])[None, :]])

  # 예측된 시작 인덱스와 종료 인덱스에 대한 값을 각각 start와 end에 저장
  start = tf.math.argmax(tf.squeeze(output[0]))
  end = tf.math.argmax(tf.squeeze(output[1])) + 1

  # 본문으로부터 예측된 '정답의 시작 인덱스'와 '종료 인덱스'를 통해 정답 추출
  answer_encoded = X_test[0][idx][start: end]
  print("예측 :", tokenizer.decode(answer_encoded))
  print('-'*50)

In [37]:
for i in range(100):
  predict_test_data_by_idx(i)

본문 : [CLS] 1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ( 폭력행위등처벌에관한법률위반 ) 으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다. 
질문 : 임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은? 
정답 : 1989년 2월 15일
예측 : 1989년 2월 15일
--------------------------------------------------
본문 : [CLS] 1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ( 폭력행위등처벌에관한법률위반 ) 으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 

### 3. 더 긴 입력을 위해 모델 바꿔보기

이 때문에 이번 실습에서는 정답 부분이 본문에서 잘리는 경우에는 해당 샘플들을 제거하여 훈련하는 방식을 택했음. 그러나 4096의 길이까지 입력을 받을 수 있는 모델을 사용하면 KorQuad 본문이 잘리는 일 없이 학습할 수 있음

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('monologg/kobigbird-bert-base')
train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

In [ ]:
with strategy.scope():
  model = TFBertForQuestionAnswering('monologg/kobigbird-bert-base')
  optimzier = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
  model.compile(optimizer=optimizer, loss=[loss, loss])

In [ ]:
train_encodings